#Import libraries needed for analysis

In [1]:
import pandas as pd
import numpy as np
%pylab inline
from sklearn.naive_bayes import GaussianNB
import pandas as pd
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


# Data import and cleaning

In [2]:
train = pd.read_csv('train.csv', header=0)
bids = pd.read_csv('bids.csv', header=0)
test = pd.read_csv('test.csv', header=0)

# Explore data

Check how many samples do we have in the datasets provided by Facebook.

In [3]:
%%bash
wc -l *.csv

 7656335 bids.csv
    4701 sampleSubmission.csv
    4701 test.csv
    2014 train.csv
 7667751 total


## Bidder dataset

In [4]:
train[:3]

,bidder_id,payment_account,address,outcome
0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0
1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0
2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0


In [5]:
train.describe()

,outcome
count,2013.000000
mean,0.051167
std,0.220394
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


Our training data consist of 2013 entries. Outcome is 1 when the bidder is a robot.

In [6]:
train[train['outcome']==1]['outcome'].count()

103

103 entries in the training dataset are robots.

In [7]:
train[train['outcome']==1][:10]

,bidder_id,payment_account,address,outcome
21,5354c02817e47f28e60e44a40ca5e48dll4sz,1862ef3b7965993cc975a1161010e810w2bwo,58e5dc4d38d886f6c7abf0c264a51cb7alhj2,1
50,9434778d2268f1fa2a8ede48c0cd05c097zey,ee3295c7b31a1e9116ebf0cf2df68ed1ac8ts,627fe7481f01372c02333477471cb237qt2vf,1
51,aabc211b4cf4d29e4ac7e7e361371622pockb,a3d2de7675556553a5f08e4c88d2c228pxtr6,a3d2de7675556553a5f08e4c88d2c228bxn98,1
58,68d468e15e0de0bab9694cdca8e799eb2j8kn,a3d2de7675556553a5f08e4c88d2c228vf08k,767150cd5a97ed796bfc58456f5500250dsv7,1
77,1cbd9081de4481ada66f1853737f7684ddjd1,a3d2de7675556553a5f08e4c88d2c228buft1,a3d2de7675556553a5f08e4c88d2c228pijoz,1
86,96b90af2d3eea4d3b2bc725e1c92e212qu52g,39ef2afbc0cde265550cafb40eb3fd8erzlrm,0e6dfa874409a90bc0fbf0d531475241cvw1o,1
139,15972a431aed117840dfce5556242366c3fwq,4b5c8f52a678e51ebe0ee3fc26da66902e2l7,865e5987e0c8fc7f6301b5571d688d13pu5j1,1
148,efc61ad6db17265a4bac85f77b154ef414tpp,a3d2de7675556553a5f08e4c88d2c22850etl,a3d2de7675556553a5f08e4c88d2c228d6pnk,1
165,3d9910a238a0f282ad3132dd6fc981c8tv3tg,a3d2de7675556553a5f08e4c88d2c2281ehah,c856c4f51e949635cb567f7321bc0c6at1y78,1
199,1af4685cda2979a608c4b6ad83473774qgicn,e35a74340ac6d90fcb4fe9b84c6eaa2135wqx,ceda5914ce08b87ec934e614452442903j17m,1


## Bid dataset

In [8]:
bids[:10]

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url
0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,9759243157894736,us,69.166.231.58,vasstdc27m7nks3
1,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,9759243157894736,in,50.201.125.84,jmqlhflrzwuay9c
2,2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,wa00e,home goods,phone2,9759243157894736,py,112.54.208.157,vasstdc27m7nks3
3,3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,jefix,jewelry,phone4,9759243157894736,in,18.99.175.133,vasstdc27m7nks3
4,4,8393c48eaf4b8fa96886edc7cf27b372dsibi,jefix,jewelry,phone5,9759243157894736,in,145.138.5.37,vasstdc27m7nks3
5,5,e8291466de91b0eb4e1515143c7f74dexy2yr,3vi4t,mobile,phone7,9759243157894736,ru,91.107.221.27,vasstdc27m7nks3
6,6,eef4c687daf977f64fc1d08675c44444raj3s,kjlzx,mobile,phone2,9759243210526315,th,152.235.155.159,j9nl1xmo6fqhcc6
7,7,ab056855c9ca9d36390feae1fa485883issyg,f5f6k,home goods,phone8,9759243210526315,id,3.210.112.183,hnt6hu93a3z1cpc
8,8,d600dc03b11e7d782e1e4dae091b084a1h5ch,h7jjx,home goods,phone9,9759243210526315,th,103.64.157.225,vasstdc27m7nks3
9,9,a58ace8b671a7531c88814bc86b2a34cf0crb,3zpkj,sporting goods,phone4,9759243210526315,za,123.28.123.226,vasstdc27m7nks3


In [9]:
bids[bids.bidder_id=='1cbd9081de4481ada66f1853737f7684ddjd1'].count()

bid_id         1103
bidder_id      1103
auction        1103
merchandise    1103
device         1103
time           1103
country        1103
ip             1103
url            1103
dtype: int64

In [10]:
bids[bids.country=='gr'][:10]

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url
709,709,cffb5f3c1f4ca815d0f643fcb2f2f316beod4,sfkrp,jewelry,phone48,9759247000000000,gr,165.99.0.12,lil9p674w7wlavk
3941,3941,e3d0a636bf384a7ea8909f392891f37emm7zb,boegs,mobile,phone615,9759264789473684,gr,62.224.154.53,vasstdc27m7nks3
4936,4936,e646db4cf8584cc9607fa26470fd0297vz3hn,boegs,sporting goods,phone698,9759267526315789,gr,91.135.63.37,0peco1g386vb4xl
7087,7087,82f35ec74e372623df8729e1c4c10b49v2zdw,jozlw,jewelry,phone124,9759272421052631,gr,158.136.24.28,4vqo5y90xsfrffz
7447,7447,e3d0a636bf384a7ea8909f392891f37emm7zb,boegs,mobile,phone11,9759273210526315,gr,214.154.90.163,953vfg16wgv3c2g
8274,8274,77f0bd275f68e66fd99b3f62669933edbzw86,spqmu,jewelry,phone854,9759275368421052,gr,11.204.67.130,7bx7acox8p1ai3e
10443,10443,e8291466de91b0eb4e1515143c7f74dexy2yr,i1aya,mobile,phone124,9759281684210526,gr,72.109.29.225,vasstdc27m7nks3
10966,10966,2271fe5b01f9b8f9d8409014ae4511524grmq,i1aya,mobile,phone49,9759283000000000,gr,138.154.61.94,2qf7ejbou2fp9u8
11176,11176,02518a364d87661eed1453e251933862yvmjj,7mkw3,home goods,phone64,9759283578947368,gr,8.112.255.168,rfgitdaealan86o
11423,11423,77f0bd275f68e66fd99b3f62669933edbzw86,spqmu,jewelry,phone854,9759284263157894,gr,11.204.67.130,7bx7acox8p1ai3e


## Exploratory data analysis

### From which 10 countries are the most bids coming from?

In [11]:
bids.country.value_counts()[:10]

in    1788731
ng     586586
id     453611
tr     319574
us     318103
za     297942
ke     287962
th     238172
my     216799
vn     190342
dtype: int64

###What merchandise is the most popular?

In [12]:
bids.merchandise.value_counts()[:10]

mobile              2126587
jewelry             1902058
sporting goods      1855207
home goods          1224234
office equipment     289838
furniture             99181
computers             81084
books and music       51941
clothing              16447
auto parts             9757
dtype: int64

### Get a list of the robots from the training set

In [13]:
train[train['outcome']==1]['bidder_id']

21     5354c02817e47f28e60e44a40ca5e48dll4sz
50     9434778d2268f1fa2a8ede48c0cd05c097zey
51     aabc211b4cf4d29e4ac7e7e361371622pockb
58     68d468e15e0de0bab9694cdca8e799eb2j8kn
77     1cbd9081de4481ada66f1853737f7684ddjd1
86     96b90af2d3eea4d3b2bc725e1c92e212qu52g
139    15972a431aed117840dfce5556242366c3fwq
148    efc61ad6db17265a4bac85f77b154ef414tpp
165    3d9910a238a0f282ad3132dd6fc981c8tv3tg
199    1af4685cda2979a608c4b6ad83473774qgicn
254    c600968815fc32b20e0519ecfce5b648en2wo
265    42c407eb70ddacdcd59d63f22caaafc4fa0p2
316    40e00fd204d0ce97110032bfc47f423d0bgrd
345    334714f005e994d46cf4d18093dd4d05fyxc8
353    a0660fe8a46b9cb6b02e7b0c50eef6a905vt6
...
1773    37bb1697e1c3922a06c2d2ea4138a4c8s6wgk
1775    c2e1242ad5ea781923a08987d55365c69fii7
1786    67a1b129ca241c51d0abb5484f8eb3fct8o5c
1843    a4e83190edb97fefdc4a6cbfd00f41fbm7c4o
1846    86e888c2dafacf13580a1211abf50961w6foa
1848    c670e7fb466b09000f1053d2761752268bkfm
1858    5f529b5ee68b60482c5d0c56ee91f3c2cjrmx

### How do the bids of the robots look like?

In [14]:
robot_bids=bids[bids.bidder_id.isin(train[train['outcome']==1]['bidder_id'])]
robot_bids

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url
20,20,af9c96944265cf541b3fe734a057821a825l7,jtxiv,office equipment,phone19,9759243263157894,hr,144.100.75.92,vasstdc27m7nks3
36,36,52d8066e28f168e5bee769559ead307e1go7e,165y9,mobile,phone33,9759243315789473,us,240.205.85.43,azl6m5i5tg78c2j
47,47,46a28eaed47c29b4fd148a50cde44840u0bsn,5h9tv,mobile,phone44,9759243421052631,in,94.210.47.16,4efsnqzirqtxasm
55,55,b1863365ae232a316eb946ba56a29415m0gdz,c8cgg,home goods,phone4,9759243473684210,vn,208.207.224.110,vasstdc27m7nks3
107,107,e12177ff9c1a8413996f7b1a590980c82ofeo,fi8mh,sporting goods,phone81,9759243736842105,uk,91.53.170.204,457usxujudk2tc9
110,110,458c8c4e10e5c1b29e3fbe5ee56d46e8y3fb3,uj7b1,sporting goods,phone53,9759243736842105,my,149.125.177.230,vasstdc27m7nks3
157,157,d2704c8bb6ebbf13e08f37131382b126wy4yc,qcppg,mobile,phone91,9759244000000000,in,37.235.109.132,a9cftgqrt3utx0c
163,163,1cbd9081de4481ada66f1853737f7684ddjd1,8bm0x,computers,phone46,9759244000000000,nl,192.22.83.50,vasstdc27m7nks3
180,180,86e888c2dafacf13580a1211abf50961w6foa,2e9us,sporting goods,phone115,9759244157894736,sg,129.223.147.141,bcdxex1pzcjieq6
229,229,c2e1242ad5ea781923a08987d55365c69fii7,idfc3,jewelry,phone142,9759244368421052,ke,41.208.146.199,7dyu8bu1q8q68h9


### Which country has the most robots, and which merchandise is the most popular on the robot bids?

In [15]:
robot_bids.country.value_counts()[:10]

in    86386
us    66663
id    20495
de    18530
th    17282
za    13364
ru    11100
mx    10848
my    10488
ca     9205
dtype: int64

In [16]:
robot_bids.merchandise.value_counts()[:10]

sporting goods      230326
mobile              105138
jewelry              37101
home goods           18708
computers            11667
office equipment      7967
books and music       1509
dtype: int64

In [17]:
gnb=GaussianNB()

In [18]:
bids_train=pd.merge(bids,train,on='bidder_id')
bids_train

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url,payment_account,address,outcome
0,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,9759243157894736,in,50.201.125.84,jmqlhflrzwuay9c,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0
1,25,668d393e858e8126275433046bbd35c6tywop,38v0a,furniture,phone4,9759243315789473,bd,147.64.206.30,vasstdc27m7nks3,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0
2,39,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone35,9759243368421052,in,75.11.239.20,i92xwb95qdo901j,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0
3,42,668d393e858e8126275433046bbd35c6tywop,jkyzt,furniture,phone38,9759243368421052,in,215.203.78.242,s4hrzaqd5vbq2he,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0
4,68,668d393e858e8126275433046bbd35c6tywop,h2xrf,furniture,phone57,9759243526315789,in,134.198.237.242,1euklxshlldto4z,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0
5,85,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone57,9759243631578947,id,78.177.183.67,oie0vw631m425f3,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0
6,140,668d393e858e8126275433046bbd35c6tywop,h2xrf,furniture,phone94,9759243947368421,in,185.104.4.204,4nhf4369ida6ugl,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0
7,168,668d393e858e8126275433046bbd35c6tywop,19fz7,furniture,phone4,9759244052631578,in,88.188.164.92,vasstdc27m7nks3,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0
8,225,668d393e858e8126275433046bbd35c6tywop,ligr3,furniture,phone3,9759244368421052,br,228.169.174.49,vasstdc27m7nks3,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0
9,239,668d393e858e8126275433046bbd35c6tywop,38v0a,furniture,phone45,9759244473684210,in,179.141.53.31,xkrb30014k3boqg,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0


In [19]:
features=bids_train.columns[0:11]

In [ ]:
y_gnb = gnb.fit(bids_train[features], bids_train['outcome'])
#.predict(bids_train[features])